In [69]:
import pandas as pd
import numpy as np
from scipy import stats

In [70]:
df = pd.read_csv('../output/voice_video_merged.csv')

/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/74949737.py:1: DtypeWarning:

Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.



In [71]:
filtered = df[df['InteractionID'].notna() & df['transactionId'].notna()].copy()

In [72]:
def normalize_iso(ts):
    if pd.isna(ts):
        return ts
    ts = str(ts)
    if ts.endswith('.000Z') and ts.count('.') > 1:
        return ts[:-4] + 'Z'     # trim the extra `.000`
    return ts

filtered['privacy_consentTimestamp_voice'] = (
    filtered['privacy_consentTimestamp_voice']
      .apply(normalize_iso)
      .pipe(pd.to_datetime, errors='coerce', utc=True)
)

/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/3875694351.py:12: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [73]:
filtered['TransactionDate'] = pd.to_datetime(filtered['TransactionDate'])
filtered['privacy_consentTimestamp_voice'] = pd.to_datetime(filtered['privacy_consentTimestamp_voice'])
filtered['TransactionDate'] = filtered['TransactionDate'].fillna(filtered['privacy_consentTimestamp_voice'])

## Extended Exploration Set-Up


In [74]:
import json
from ast import literal_eval
import plotly.express as px
import plotly.graph_objects as go


def safe_json(value, default):
    """Convert mixed JSON strings/lists/dicts into python objects."""
    if pd.isna(value):
        return default
    if isinstance(value, (dict, list)):
        return value
    text = str(value).strip()
    if not text:
        return default
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            return literal_eval(text)
        except (ValueError, SyntaxError):
            return default


def ensure_datetime(series):
    """Standardize datetime with UTC awareness."""
    return pd.to_datetime(series, errors='coerce', utc=True)



In [75]:
# --- data hygiene -----------------------------------------------------------

filtered['TransactionDate'] = ensure_datetime(filtered['TransactionDate'])
filtered['privacy_consentTimestamp_voice'] = ensure_datetime(filtered['privacy_consentTimestamp_voice'])
filtered['TransactionDate'] = filtered['TransactionDate'].fillna(filtered['privacy_consentTimestamp_voice'])

# standardize gender entries (strip stray quotes) then drop unknowns
filtered['gender_clean'] = (
    filtered['Gender']
    .astype(str)
    .str.strip()
    .str.strip("'\"")
    .str.title()
)
filtered['gender_clean'] = filtered['gender_clean'].where(filtered['gender_clean'].isin(['Male', 'Female']))
filtered = filtered.dropna(subset=['TransactionDate', 'gender_clean'])

# numeric age + bucket
filtered['Age'] = pd.to_numeric(filtered['Age'], errors='coerce')
age_bins = [0, 17, 24, 34, 44, 54, 120]
age_labels = ['<18', '18-24', '25-34', '35-44', '45-54', '55+']
filtered['age_bucket'] = pd.cut(filtered['Age'], bins=age_bins, labels=age_labels, right=True)

# convenience date columns
filtered['txn_date'] = filtered['TransactionDate'].dt.date
filtered['txn_month'] = filtered['TransactionDate'].dt.to_period('M').dt.to_timestamp()
filtered['txn_weekday'] = filtered['TransactionDate'].dt.day_name()
filtered['txn_hour'] = filtered['TransactionDate'].dt.hour



/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/3657606100.py:26: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [76]:
# --- item-level expansion for product analytics ----------------------------

items_series = filtered['items_voice'].apply(lambda x: safe_json(x, []))
items_df = filtered[['InteractionID', 'TransactionDate', 'gender_clean', 'age_bucket', 'Age', 'transactionContext_paymentMethod_voice', 'totals_totalAmount_voice']].copy()
items_df['items'] = items_series
items_df = items_df.explode('items').dropna(subset=['items'])
item_details = pd.json_normalize(items_df['items'])
items_df = pd.concat([items_df.reset_index(drop=True), item_details], axis=1).drop(columns=['items'])

for col in ['totalPrice', 'unitPrice', 'quantity']:
    if col not in items_df.columns:
        items_df[col] = np.nan
items_df['totalPrice'] = pd.to_numeric(items_df['totalPrice'], errors='coerce')
items_df['unitPrice'] = pd.to_numeric(items_df['unitPrice'], errors='coerce')
items_df['quantity'] = pd.to_numeric(items_df['quantity'], errors='coerce')
for text_col in ['category', 'brandName', 'productName', 'sku']:
    if text_col not in items_df.columns:
        items_df[text_col] = np.nan
items_df['category'] = items_df['category'].fillna('Unspecified')
items_df['brandName'] = items_df['brandName'].fillna('Unspecified')

# Additional cleaning: convert items with brandName=='TM' to category=='Services'
items_df.loc[items_df['brandName'] == 'TM', 'category'] = 'Services'

# helper for time-of-day segments
# Late Night (10p-5a) covers hours 22-24 and 0-5
def get_timeofday_segment(hour):
    if hour >= 22 or hour < 5:
        return 'Late Night (10p-5a)'
    elif hour >= 5 and hour < 12:
        return 'Morning (5a-12p)'
    elif hour >= 12 and hour < 18:
        return 'Afternoon (12p-6p)'
    elif hour >= 18 and hour < 22:
        return 'Evening (6p-10p)'
    else:
        return 'Late Night (10p-5a)'

filtered['timeofday_segment'] = filtered['txn_hour'].apply(get_timeofday_segment)
items_df = items_df.merge(filtered[['InteractionID', 'timeofday_segment', 'txn_weekday']], on='InteractionID', how='left')

# Drop outliers where basket_total (totals_totalAmount_voice) > 6000
print(f"Before filtering: {len(items_df)} rows")
items_df = items_df[items_df['totals_totalAmount_voice'] <= 6000].copy()
print(f"After filtering (basket_total <= 6000): {len(items_df)} rows")



Before filtering: 22622 rows
After filtering (basket_total <= 6000): 22613 rows


In [77]:
items_df[items_df['brandName']=='TM']['category'].value_counts()

category
Services    159
Name: count, dtype: int64

In [78]:
items_df[items_df['category']=='Services']['brandName'].value_counts()

brandName
Globe             289
Cherry Prepaid    285
TM                159
Name: count, dtype: int64

In [79]:
items_df['category'].value_counts()

category
Snacks & Confectionery           2418
Other Essentials                 2182
unspecified                      2115
Beverages                        1704
Laundry                          1637
Non-Alcoholic                    1515
Hair Care                        1502
Canned & Jarred Goods            1214
Pantry Staples & Groceries       1081
Cooking Essentials                942
Body Care                         932
Instant Foods                     889
Oral Care                         867
Biscuits & Crackers               820
Salty Snacks (Chichirya)          813
Tobacco Products                  734
Services                          733
Alcoholic                         168
Food & Beverages                  128
Candies & Sweets                  119
Household & Cleaning Supplies      99
Personal Care & Hygiene             1
Name: count, dtype: int64

In [80]:
## Outlier Detection for Average Metrics

# Columns used in average calculations that need outlier checking
# 1. totals_totalAmount_voice (basket_total) - used for avg_spend
# 2. totalPrice, unitPrice, quantity - used for item-level averages

def detect_outliers_iqr(df, column, multiplier=1.5):
    """Detect outliers using IQR method."""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound, Q1, Q3, IQR

def detect_outliers_zscore(df, column, threshold=3):
    """Detect outliers using Z-score method."""
    from scipy import stats
    z_scores = np.abs(stats.zscore(df[column].dropna()))
    outlier_mask = z_scores > threshold
    outliers = df[outlier_mask]
    return outliers, z_scores

def print_outlier_summary(df, column, outliers_iqr, outliers_zscore, lower_bound, upper_bound):
    """Print summary statistics for outliers."""
    print(f"\n{'='*60}")
    print(f"OUTLIER ANALYSIS: {column}")
    print(f"{'='*60}")
    print(f"\nTotal records: {len(df):,}")
    print(f"Non-null values: {df[column].notna().sum():,}")
    print(f"\nBasic Statistics:")
    print(f"  Mean: ₱{df[column].mean():,.2f}")
    print(f"  Median: ₱{df[column].median():,.2f}")
    print(f"  Std Dev: ₱{df[column].std():,.2f}")
    print(f"  Min: ₱{df[column].min():,.2f}")
    print(f"  Max: ₱{df[column].max():,.2f}")
    print(f"\nIQR Method Outliers (1.5x IQR):")
    print(f"  Lower bound: ₱{lower_bound:,.2f}")
    print(f"  Upper bound: ₱{upper_bound:,.2f}")
    print(f"  Outlier count: {len(outliers_iqr):,} ({len(outliers_iqr)/len(df)*100:.2f}%)")
    print(f"\nZ-Score Method Outliers (|z| > 3):")
    print(f"  Outlier count: {len(outliers_zscore):,} ({len(outliers_zscore)/len(df)*100:.2f}%)")
    
    if len(outliers_iqr) > 0:
        print(f"\nTop 10 IQR Outliers:")
        top_outliers = outliers_iqr.nlargest(10, column)[['InteractionID', column, 'TransactionDate', 'gender_clean', 'age_bucket']]
        print(top_outliers.to_string(index=False))

# 1. Outlier detection for basket_total (totals_totalAmount_voice)
print("\n" + "="*60)
print("BASKET TOTAL OUTLIER DETECTION")
print("="*60)

# Convert to numeric if not already
filtered['totals_totalAmount_voice'] = pd.to_numeric(filtered['totals_totalAmount_voice'], errors='coerce')
basket_data = filtered[filtered['totals_totalAmount_voice'].notna()].copy()

if len(basket_data) > 0:
    outliers_iqr, lower_bound, upper_bound, Q1, Q3, IQR = detect_outliers_iqr(
        basket_data, 'totals_totalAmount_voice', multiplier=1.5
    )
    outliers_zscore, z_scores = detect_outliers_zscore(
        basket_data, 'totals_totalAmount_voice', threshold=3
    )
    
    print_outlier_summary(
        basket_data, 'totals_totalAmount_voice', 
        outliers_iqr, outliers_zscore, lower_bound, upper_bound
    )
    
    # Visualization: Box plot
    fig_box = go.Figure()
    fig_box.add_trace(go.Box(
        y=basket_data['totals_totalAmount_voice'],
        name='Basket Total',
        boxmean='sd'
    ))
    fig_box.update_layout(
        title='Basket Total Distribution (Box Plot)',
        yaxis_title='Basket Total (₱)',
        showlegend=False
    )
    fig_box.show()
    
    # Visualization: Histogram with outlier markers
    fig_hist = go.Figure()
    fig_hist.add_trace(go.Histogram(
        x=basket_data['totals_totalAmount_voice'],
        name='All Data',
        nbinsx=50
    ))
    if len(outliers_iqr) > 0:
        fig_hist.add_trace(go.Histogram(
            x=outliers_iqr['totals_totalAmount_voice'],
            name='IQR Outliers',
            nbinsx=50,
            marker_color='red',
            opacity=0.7
        ))
    fig_hist.update_layout(
        title='Basket Total Distribution with Outliers',
        xaxis_title='Basket Total (₱)',
        yaxis_title='Frequency',
        barmode='overlay'
    )
    fig_hist.show()
    
    # Visualization: Scatter plot of outliers over time
    if len(outliers_iqr) > 0:
        fig_scatter = go.Figure()
        fig_scatter.add_trace(go.Scatter(
            x=basket_data['TransactionDate'],
            y=basket_data['totals_totalAmount_voice'],
            mode='markers',
            name='Normal',
            marker=dict(color='blue', size=4, opacity=0.3)
        ))
        fig_scatter.add_trace(go.Scatter(
            x=outliers_iqr['TransactionDate'],
            y=outliers_iqr['totals_totalAmount_voice'],
            mode='markers',
            name='Outliers',
            marker=dict(color='red', size=8, opacity=0.8)
        ))
        fig_scatter.update_layout(
            title='Basket Total Over Time (Outliers Highlighted)',
            xaxis_title='Transaction Date',
            yaxis_title='Basket Total (₱)'
        )
        fig_scatter.show()

# 2. Outlier detection for item-level metrics (totalPrice, unitPrice, quantity)
print("\n" + "="*60)
print("ITEM-LEVEL METRICS OUTLIER DETECTION")
print("="*60)

item_metrics = ['totalPrice', 'unitPrice', 'quantity']
for metric in item_metrics:
    if metric in items_df.columns:
        metric_data = items_df[items_df[metric].notna()].copy()
        
        if len(metric_data) > 0:
            outliers_iqr, lower_bound, upper_bound, Q1, Q3, IQR = detect_outliers_iqr(
                metric_data, metric, multiplier=1.5
            )
            outliers_zscore, z_scores = detect_outliers_zscore(
                metric_data, metric, threshold=3
            )
            
            print_outlier_summary(
                metric_data, metric,
                outliers_iqr, outliers_zscore, lower_bound, upper_bound
            )
            
            # Visualization: Box plot for each metric
            fig_box = go.Figure()
            fig_box.add_trace(go.Box(
                y=metric_data[metric],
                name=metric,
                boxmean='sd'
            ))
            fig_box.update_layout(
                title=f'{metric} Distribution (Box Plot)',
                yaxis_title=f'{metric}',
                showlegend=False
            )
            fig_box.show()

print("\n" + "="*60)
print("OUTLIER DETECTION COMPLETE")
print("="*60)
print("\nRecommendations:")
print("1. Review outliers to determine if they are data errors or legitimate extreme values")
print("2. Consider capping outliers at reasonable thresholds if they skew averages")
print("3. Document any data quality issues found")
print("4. Consider using median instead of mean for metrics with many outliers")




BASKET TOTAL OUTLIER DETECTION

OUTLIER ANALYSIS: totals_totalAmount_voice

Total records: 11,892
Non-null values: 11,892

Basic Statistics:
  Mean: ₱113.05
  Median: ₱45.00
  Std Dev: ₱251.48
  Min: ₱6.00
  Max: ₱12,000.00

IQR Method Outliers (1.5x IQR):
  Lower bound: ₱-131.09
  Upper bound: ₱278.66
  Outlier count: 1,149 (9.66%)

Z-Score Method Outliers (|z| > 3):
  Outlier count: 84 (0.71%)

Top 10 IQR Outliers:
                       InteractionID  totals_totalAmount_voice           TransactionDate gender_clean age_bucket
4bc3ab70-d46e-4669-a74a-c2596da70392                  12000.00 2025-09-20 01:00:29+00:00         Male      35-44
e3b6ec75-62a0-461b-b50d-be8042cccbce                  10048.83 2025-07-29 10:27:45+00:00         Male      35-44
bf6ddc51-1d11-4a9e-a695-b05d54925799                  10000.00 2025-07-08 00:29:32+00:00       Female      35-44
864c903c-0c36-45be-be48-a6389264e644                   6435.00 2025-07-14 12:24:10+00:00       Female      35-44
f0ef682d-8855


ITEM-LEVEL METRICS OUTLIER DETECTION

OUTLIER ANALYSIS: totalPrice

Total records: 22,613
Non-null values: 22,613

Basic Statistics:
  Mean: ₱73.68
  Median: ₱32.00
  Std Dev: ₱123.44
  Min: ₱6.00
  Max: ₱5,000.00

IQR Method Outliers (1.5x IQR):
  Lower bound: ₱-81.75
  Upper bound: ₱185.05
  Outlier count: 1,989 (8.80%)

Z-Score Method Outliers (|z| > 3):
  Outlier count: 337 (1.49%)

Top 10 IQR Outliers:
                       InteractionID  totalPrice           TransactionDate gender_clean age_bucket
f0ef682d-8855-4abf-8d4b-a5bdcd7468fe      5000.0 2025-08-07 08:44:08+00:00         Male      35-44
ac568d20-952c-410e-94a8-a9597d583e91      4000.0 2025-07-12 08:41:19+00:00         Male      45-54
ac568d20-952c-410e-94a8-a9597d583e91      4000.0 2025-07-12 08:41:19+00:00         Male      45-54
d35164b6-8a03-465e-9dd2-7ccd37b910aa      3500.1 2025-07-30 14:12:42+00:00       Female      35-44
d35164b6-8a03-465e-9dd2-7ccd37b910aa      3500.1 2025-07-30 14:12:42+00:00       Female      


OUTLIER ANALYSIS: unitPrice

Total records: 22,613
Non-null values: 22,613

Basic Statistics:
  Mean: ₱45.49
  Median: ₱27.30
  Std Dev: ₱42.97
  Min: ₱6.00
  Max: ₱210.00

IQR Method Outliers (1.5x IQR):
  Lower bound: ₱-62.75
  Upper bound: ₱147.25
  Outlier count: 805 (3.56%)

Z-Score Method Outliers (|z| > 3):
  Outlier count: 799 (3.53%)

Top 10 IQR Outliers:
                       InteractionID  unitPrice           TransactionDate gender_clean age_bucket
06562073-72ff-45f9-ab50-43c206a6797c      210.0 2025-06-21 05:21:40+00:00         Male      35-44
06562073-72ff-45f9-ab50-43c206a6797c      210.0 2025-06-21 05:21:40+00:00         Male      35-44
0fee0c1b-5f1d-4a1c-8ed4-e93ab031fee6      210.0 2025-05-24 01:16:57+00:00         Male      35-44
0fee0c1b-5f1d-4a1c-8ed4-e93ab031fee6      210.0 2025-05-24 01:16:57+00:00         Male      35-44
1386729b-0de0-4eb1-8613-5ace85539297      210.0 2025-07-06 13:00:42+00:00       Female      18-24
174cbcd2-8975-4514-9f07-a68c61903a25      21


OUTLIER ANALYSIS: quantity

Total records: 22,613
Non-null values: 22,613

Basic Statistics:
  Mean: ₱1.51
  Median: ₱1.00
  Std Dev: ₱1.47
  Min: ₱1.00
  Max: ₱95.00

IQR Method Outliers (1.5x IQR):
  Lower bound: ₱-0.50
  Upper bound: ₱3.50
  Outlier count: 275 (1.22%)

Z-Score Method Outliers (|z| > 3):
  Outlier count: 122 (0.54%)

Top 10 IQR Outliers:
                       InteractionID  quantity           TransactionDate gender_clean age_bucket
e4ab015a-b88c-4780-a672-a38b610df1ea        95 2025-07-08 10:48:05+00:00         Male      25-34
f0ef682d-8855-4abf-8d4b-a5bdcd7468fe        50 2025-08-07 08:44:08+00:00         Male      35-44
29611b5c-53a4-43d4-970e-5beb6a36d32c        45 2025-05-04 12:06:11+00:00         Male      35-44
ca1d2b1e-37b2-401e-8f8e-231d2ae03a02        45 2025-07-31 12:51:38+00:00         Male      35-44
ca1d2b1e-37b2-401e-8f8e-231d2ae03a02        45 2025-07-31 12:51:38+00:00         Male      35-44
d35164b6-8a03-465e-9dd2-7ccd37b910aa        45 2025-07-30 


OUTLIER DETECTION COMPLETE

Recommendations:
1. Review outliers to determine if they are data errors or legitimate extreme values
2. Consider capping outliers at reasonable thresholds if they skew averages
3. Document any data quality issues found
4. Consider using median instead of mean for metrics with many outliers


In [81]:
# --- final flattened transaction-level table for analytics / BI exports ---

final_df = filtered[[
    "InteractionID",
    "TransactionDate",
    "txn_date",
    "txn_month",
    "txn_weekday",
    "txn_hour",
    "timeofday_segment",
    "Gender",
    "gender_clean",
    "Age",
    "age_bucket",
    "transactionContext_paymentMethod_voice",
    "totals_totalAmount_voice",
]].copy()

final_df.rename(
    columns={
        "transactionContext_paymentMethod_voice": "payment_method",
        "totals_totalAmount_voice": "basket_total",
    },
    inplace=True,
)

# Drop outliers where basket_total > 6000
print(f"Before filtering: {len(final_df)} rows")
final_df = final_df[final_df['basket_total'] <= 6000].copy()
print(f"After filtering (basket_total <= 6000): {len(final_df)} rows")

final_df.head()


Before filtering: 11892 rows
After filtering (basket_total <= 6000): 11888 rows


,InteractionID,TransactionDate,txn_date,txn_month,txn_weekday,txn_hour,timeofday_segment,Gender,gender_clean,Age,age_bucket,payment_method,basket_total
31,000cef26-91c2-494e-8d81-fb602dc28906,2025-05-29 04:38:52+00:00,2025-05-29,2025-05-01,Thursday,4,Late Night (10p-5a),'Male',Male,36.0,35-44,cash,336.00
44,000f9d0e-5c9b-4f5d-92e9-3bd60de578ce,2025-10-04 13:39:31+00:00,2025-10-04,2025-10-01,Saturday,13,Afternoon (12p-6p),'Female',Female,39.0,35-44,cash,42.75
128,002f5ec0-3f50-4403-b84c-7fcff5d44f3a,2025-07-15 04:50:18+00:00,2025-07-15,2025-07-01,Tuesday,4,Late Night (10p-5a),'Female',Female,40.0,35-44,cash,25.45
177,0043f3a5-a2b3-469a-a330-4bce6f24b845,2025-06-02 04:23:50+00:00,2025-06-02,2025-06-01,Monday,4,Late Night (10p-5a),'Female',Female,33.0,25-34,cash,10.00
196,004a0c1b-bbb5-4f55-824f-4c9f3bbea4f6,2025-05-19 10:24:43+00:00,2025-05-19,2025-05-01,Monday,10,Morning (5a-12p),'Female',Female,36.0,35-44,cash,100.00


## Demographics — Gender View


In [ ]:
gender_summary = (
    filtered.groupby('gender_clean')
      .agg(total_transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'))
      .reset_index()
)

fig_gender_txn = px.bar(
    gender_summary,
    x='gender_clean',
    y='total_transactions',
    color='gender_clean',
    title='Total Transactions by Gender',
    labels={'gender_clean': 'Gender', 'total_transactions': 'Transactions'},
    text_auto=True
)
fig_gender_txn.update_layout(showlegend=False)

fig_gender_avg = px.bar(
    gender_summary,
    x='gender_clean',
    y='avg_spend',
    color='gender_clean',
    title='Average Basket Spend by Gender',
    labels={'avg_spend': 'Average Spend (₱)'},
    text_auto='.2f'
)
fig_gender_avg.update_layout(showlegend=False)

monthly_gender = (
    filtered.groupby(['txn_month', 'gender_clean'])
      .agg(total_transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'))
      .reset_index()
)

fig_gender_mom = px.line(
    monthly_gender,
    x='txn_month',
    y='total_transactions',
    color='gender_clean',
    markers=True,
    title='Month-on-Month Transactions by Gender',
    labels={'txn_month': 'Month', 'total_transactions': 'Transactions'}
)

fig_gender_txn.show()
fig_gender_avg.show()
fig_gender_mom.show()



## Demographics — Age Buckets


In [83]:
age_summary = (
    filtered.dropna(subset=['age_bucket'])
      .groupby('age_bucket')
      .agg(total_transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'))
      .reset_index()
)

fig_age = px.bar(
    age_summary,
    x='age_bucket',
    y='total_transactions',
    color='avg_spend',
    title='Transactions & Avg Spend by Age Bucket',
    labels={'age_bucket': 'Age Bucket', 'total_transactions': 'Transactions', 'avg_spend': 'Avg Spend'},
    text='avg_spend',
    color_continuous_scale='Blues'
)
fig_age.update_traces(texttemplate='₱%{text:.0f}', textposition='outside')
fig_age.show()



/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/3048095839.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Demographics — Tender Type (Cash vs E-Wallet)


In [84]:
filtered.columns

Index(['InteractionID', 'StoreID', 'ProductID', 'TransactionDate', 'DeviceID',
       'FacialID', 'Sex', 'Age', 'EmotionalState', 'TranscriptionText',
       'Gender', 'Barangay', 'canonical_tx_id_norm', 'canonical_tx_id',
       'storeId_voice', 'deviceId_voice', 'timestamp_voice', 'transactionId',
       'brandDetection_voice', 'items_voice', 'totals_voice',
       'transactionContext_voice', 'privacy_voice', 'processingTime_voice',
       'edgeVersion_voice', '_file_path_voice', 'privacy_audioStored_voice',
       'privacy_brandAnalysisOnly_voice', 'privacy_noFacialRecognition_voice',
       'privacy_noImageProcessing_voice', 'privacy_dataRetentionDays_voice',
       'privacy_anonymizationLevel_voice', 'privacy_consentTimestamp_voice',
       'totals_totalAmount_voice', 'totals_totalItems_voice',
       'totals_brandedAmount_voice', 'totals_unbrandedAmount_voice',
       'totals_brandedCount_voice', 'totals_unbrandedCount_voice',
       'totals_uniqueBrandsCount_voice', 'transaction

In [85]:
tender_summary = (
    filtered.groupby('transactionContext_paymentMethod_voice')
      .agg(total_transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'))
      .reset_index()
)

fig_tender = px.bar(
    tender_summary,
    x='transactionContext_paymentMethod_voice',
    y='total_transactions',
    color='transactionContext_paymentMethod_voice',
    title='Transactions by Tender Type',
    text_auto=True,
    labels={'transactionContext_paymentMethod_voice': 'Payment Method', 'total_transactions': 'Transactions'}
)
fig_tender.update_layout(showlegend=False)

fig_tender_avg = px.scatter(
    tender_summary,
    x='transactionContext_paymentMethod_voice',
    y='avg_spend',
    size='total_transactions',
    color='avg_spend',
    title='Average Spend by Tender Type',
    labels={'avg_spend': 'Average Spend (₱)'},
    color_continuous_scale='Viridis'
)

fig_tender.show()
fig_tender_avg.show()



## Shopping Behavior — Weekday / Weekend & Time of Day


In [86]:
filtered['weekday_type'] = np.where(filtered['TransactionDate'].dt.dayofweek >= 5, 'Weekend', 'Weekday')

week_summary = (
    filtered.groupby('weekday_type')
      .agg(total_transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'))
      .reset_index()
)

fig_week = px.bar(
    week_summary,
    x='weekday_type',
    y='total_transactions',
    color='weekday_type',
    text_auto=True,
    title='Transactions: Weekday vs Weekend'
)
fig_week.update_layout(showlegend=False)

fig_week_avg = px.bar(
    week_summary,
    x='weekday_type',
    y='avg_spend',
    color='weekday_type',
    text_auto='.2f',
    title='Average Spend: Weekday vs Weekend'
)
fig_week_avg.update_layout(showlegend=False)

timeofday_summary = (
    filtered.dropna(subset=['timeofday_segment'])
      .groupby(['weekday_type', 'timeofday_segment'])
      .agg(total_transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'))
      .reset_index()
)

fig_time = px.bar(
    timeofday_summary,
    x='timeofday_segment',
    y='total_transactions',
    color='weekday_type',
    barmode='group',
    title='Transactions by Time of Day (Weekday vs Weekend)',
    labels={'timeofday_segment': 'Time of Day'}
)

fig_time_avg = px.line(
    timeofday_summary,
    x='timeofday_segment',
    y='avg_spend',
    color='weekday_type',
    markers=True,
    title='Average Spend by Time of Day'
)

for fig in [fig_week, fig_week_avg, fig_time, fig_time_avg]:
    fig.show()



## Shopping Behavior — Payday Windows


In [87]:
payday_days = {12, 13, 14, 27, 28, 29}
filtered['payday_window'] = np.where(filtered['TransactionDate'].dt.day.isin(payday_days), 'Payday Window', 'Rest of Month')

payday_summary = (
    filtered.groupby('payday_window')
      .agg(total_transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'),
           total_revenue=('totals_totalAmount_voice', 'sum'))
      .reset_index()
)

fig_payday = px.bar(
    payday_summary,
    x='payday_window',
    y='total_transactions',
    color='payday_window',
    text_auto=True,
    title='Transactions During Payday Windows'
)
fig_payday.update_layout(showlegend=False)

fig_payday_avg = px.bar(
    payday_summary,
    x='payday_window',
    y='avg_spend',
    color='payday_window',
    text_auto='.2f',
    title='Average Spend During Payday Windows'
)
fig_payday_avg.update_layout(showlegend=False)

fig_payday_rev = px.pie(
    payday_summary,
    names='payday_window',
    values='total_revenue',
    title='Revenue Split — Payday vs Rest of Month'
)

fig_payday.show()
fig_payday_avg.show()
fig_payday_rev.show()



## Basket Value & Price-Point Signals


In [88]:
basket_bins = [0, 10, 20, 50, 100, 200, np.inf]
basket_labels = ['₱0-10', '₱11-20', '₱21-50', '₱51-100', '₱101-200', '₱200+']
filtered['basket_band'] = pd.cut(filtered['totals_totalAmount_voice'], bins=basket_bins, labels=basket_labels, right=True)

basket_summary = (
    filtered.dropna(subset=['basket_band'])
      .groupby('basket_band')
      .agg(transactions=('InteractionID', 'count'),
           avg_spend=('totals_totalAmount_voice', 'mean'))
      .reset_index()
)

fig_basket = px.bar(
    basket_summary,
    x='basket_band',
    y='transactions',
    color='avg_spend',
    title='Basket Value Distribution',
    text='transactions',
    labels={'basket_band': 'Basket Band', 'transactions': 'Transactions'},
    color_continuous_scale='Tealgrn'
)
fig_basket.show()

# SKU price points at 5/10/15 pesos (±0.25 tolerance)
round_targets = [5, 10, 15]
tolerance = 0.25

def closest_round(value):
    if pd.isna(value):
        return None
    for target in round_targets:
        if abs(value - target) <= tolerance:
            return f'₱{target}'
    return None

items_df['round_price_flag'] = items_df['unitPrice'].apply(closest_round)
round_summary = (
    items_df.dropna(subset=['round_price_flag'])
            .groupby('round_price_flag')
            .agg(freq=('sku', 'count'))
            .reset_index()
)

fig_round = px.pie(
    round_summary,
    names='round_price_flag',
    values='freq',
    title='Share of Items Sold at ₱5 / ₱10 / ₱15',
    hole=0.35
)

fig_round.show()



/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/1863763441.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Top Products by Daypart


In [89]:
daypart_products = (
    items_df.dropna(subset=['timeofday_segment'])
            .groupby(['timeofday_segment', 'brandName'])
            .agg(quantity=('quantity', 'sum'),
                 revenue=('totalPrice', 'sum'))
            .reset_index()
)

# keep top 5 per segment
ranked = daypart_products.sort_values(['timeofday_segment', 'quantity'], ascending=[True, False])
ranked['rank'] = ranked.groupby('timeofday_segment')['quantity'].rank(method='first', ascending=False)
top_daypart = ranked[ranked['rank'] <= 5]

fig_daypart = px.bar(
    top_daypart,
    x='brandName',
    y='quantity',
    color='revenue',
    facet_col='timeofday_segment',
    facet_col_wrap=2,
    title='Top 5 Brands per Daypart',
    labels={'brandName': 'Brand', 'quantity': 'Qty', 'revenue': 'Revenue'},
    color_continuous_scale='OrRd'
)
fig_daypart.for_each_xaxis(lambda ax: ax.update(tickangle=45))
fig_daypart.update_layout(height=600)
fig_daypart.show()



## Category Performance Breakdown


In [90]:
category_summary = (
    items_df.groupby('category')
            .agg(units=('quantity', 'sum'),
                 revenue=('totalPrice', 'sum'))
            .reset_index()
            .sort_values('revenue', ascending=False)
)

fig_cat_rank = px.bar(
    category_summary.head(15),
    x='category',
    y='revenue',
    text='units',
    title='Top Categories by Revenue (text = units)',
    labels={'revenue': 'Revenue', 'category': 'Category'}
)
fig_cat_rank.update_traces(texttemplate='%{text:.0f} units', textposition='outside')
fig_cat_rank.update_layout(xaxis_tickangle=45)
fig_cat_rank.show()

category_by_day = (
    items_df.groupby(['txn_weekday', 'category'])
            .agg(units=('quantity', 'sum'))
            .reset_index()
)
heat_day = category_by_day.pivot(index='txn_weekday', columns='category', values='units').fillna(0)
fig_cat_day = px.imshow(
    heat_day,
    aspect='auto',
    color_continuous_scale='Purples',
    title='Category Units by Day of Week'
)
fig_cat_day.show()

category_by_gender = (
    items_df.groupby(['gender_clean', 'category'])
            .agg(units=('quantity', 'sum'))
            .reset_index()
)
fig_cat_gender = px.bar(
    category_by_gender,
    x='category',
    y='units',
    color='gender_clean',
    barmode='group',
    title='Category Units by Gender'
)
fig_cat_gender.update_layout(xaxis_tickangle=45)
fig_cat_gender.show()

category_by_age = (
    items_df.dropna(subset=['age_bucket'])
            .groupby(['age_bucket', 'category'])
            .agg(units=('quantity', 'sum'))
            .reset_index()
)
fig_cat_age = px.bar(
    category_by_age,
    x='age_bucket',
    y='units',
    color='category',
    title='Category Mix by Age Bucket',
    labels={'age_bucket': 'Age Bucket'},
    barmode='stack'
)
fig_cat_age.show()



/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/7780318.py:53: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Category Composition by Basket Band


In [91]:
items_with_bands = items_df.merge(filtered[['InteractionID', 'basket_band']], on='InteractionID', how='left')
composition = (
    items_with_bands.dropna(subset=['basket_band'])
                    .groupby(['basket_band', 'category'])
                    .agg(revenue=('totalPrice', 'sum'))
                    .reset_index()
)

fig_comp = px.bar(
    composition,
    x='basket_band',
    y='revenue',
    color='category',
    title='Category Composition within Basket Bands',
    labels={'basket_band': 'Basket Band', 'revenue': 'Revenue'},
    barmode='stack'
)
fig_comp.show()



/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/2890242115.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Frequently Paired Products


In [92]:
from itertools import combinations

pairs = (
    items_df.groupby('InteractionID')['brandName']
            .apply(lambda brands: list(combinations(sorted(set([b for b in brands if isinstance(b, str) and b.strip()])), 2)))
            .explode()
            .dropna()
)

# Count pairs and convert to DataFrame
pair_counts = pairs.value_counts().reset_index()
# The first column contains the pair tuples, rename it
pair_counts.columns = ['pair', 'frequency']

# Extract Brand A and Brand B from the pair tuples
pair_counts[['Brand A', 'Brand B']] = pd.DataFrame(pair_counts['pair'].tolist(), index=pair_counts.index)

fig_pairs = px.bar(
    pair_counts.head(15),
    x='frequency',
    y='Brand A',
    color='Brand B',
    orientation='h',
    title='Top Product Pairings',
    labels={'frequency': 'Co-occurrences'}
)
fig_pairs.show()


## Tobacco Analysis


In [93]:
tobacco_keywords = ['tobacco', 'cig', 'marlboro', 'fort', 'chester', 'winston', 'hope']

def is_tobacco(row):
    text = ' '.join([
        str(row.get('category', '')),
        str(row.get('brandName', '')),
        str(row.get('productName', ''))
    ]).lower()
    return any(keyword in text for keyword in tobacco_keywords)

tobacco_df = items_df[items_df.apply(is_tobacco, axis=1)].copy()

if not tobacco_df.empty:
    tobacco_df['hour'] = tobacco_df['TransactionDate'].dt.hour
    tobacco_df['weekday'] = tobacco_df['TransactionDate'].dt.day_name()

    time_qty = (
        tobacco_df.groupby('hour')
                  .agg(total_qty=('quantity', 'sum'),
                       avg_qty=('quantity', 'mean'))
                  .reset_index()
    )
    fig_tob_time = px.line(
        time_qty,
        x='hour',
        y='total_qty',
        markers=True,
        title='Tobacco Volume by Hour of Day',
        labels={'hour': 'Hour', 'total_qty': 'Qty'}
    )
    fig_tob_time.add_trace(go.Bar(x=time_qty['hour'], y=time_qty['avg_qty'], name='Avg Qty', opacity=0.3))
    fig_tob_time.show()

    brand_summary = (
        tobacco_df.groupby('brandName')
                  .agg(total_qty=('quantity', 'sum'),
                       revenue=('totalPrice', 'sum'))
                  .reset_index()
                  .sort_values('total_qty', ascending=False)
    )
    fig_tob_brand = px.bar(
        brand_summary.head(10),
        x='brandName',
        y='total_qty',
        color='revenue',
        title='Top Tobacco Brands',
        labels={'brandName': 'Brand', 'total_qty': 'Qty'}
    )
    fig_tob_brand.show()

    brand_day = (
        tobacco_df.groupby(['weekday', 'brandName'])
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_tob_brand_day = px.bar(
        brand_day,
        x='weekday',
        y='total_qty',
        color='brandName',
        title='Tobacco Qty by Day x Brand',
        barmode='stack'
    )
    fig_tob_brand_day.show()

    day_avg = (
        tobacco_df.groupby('weekday')
                  .agg(avg_qty=('quantity', 'mean'))
                  .reset_index()
    )
    fig_tob_day_avg = px.bar(
        day_avg,
        x='weekday',
        y='avg_qty',
        title='Average Tobacco Qty per Transaction by Day'
    )
    fig_tob_day_avg.show()
else:
    print('No tobacco records detected — adjust keyword list if needed.')



### Tobacco — Demographic Splits


In [94]:
if not tobacco_df.empty:
    tob_gender = (
        tobacco_df.groupby('gender_clean')
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_tob_gender = px.bar(
        tob_gender,
        x='gender_clean',
        y='total_qty',
        title='Tobacco Qty by Gender',
        text_auto=True
    )
    fig_tob_gender.show()

    tob_age = (
        tobacco_df.dropna(subset=['age_bucket'])
                  .groupby('age_bucket')
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_tob_age = px.bar(
        tob_age,
        x='age_bucket',
        y='total_qty',
        title='Tobacco Qty by Age Bucket'
    )
    fig_tob_age.show()

    tob_gender_brand = (
        tobacco_df.groupby(['gender_clean', 'brandName'])
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_tob_gender_brand = px.bar(
        tob_gender_brand,
        x='brandName',
        y='total_qty',
        color='gender_clean',
        title='Tobacco Brands by Gender',
        barmode='group'
    )
    fig_tob_gender_brand.update_layout(xaxis_tickangle=45)
    fig_tob_gender_brand.show()



/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/2189214065.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Marlboro Basket Attachments


In [95]:
marlboro_txn_ids = items_df[items_df['brandName'].str.contains('marlboro', case=False, na=False)]['InteractionID'].unique()
marlboro_baskets = items_df[items_df['InteractionID'].isin(marlboro_txn_ids)]

co_brands = (
    marlboro_baskets[~marlboro_baskets['brandName'].str.contains('marlboro', case=False, na=False)]
                 .groupby('brandName')
                 .agg(co_occurrences=('InteractionID', 'nunique'),
                      units=('quantity', 'sum'),
                      revenue=('totalPrice', 'sum'))
                 .reset_index()
                 .sort_values('co_occurrences', ascending=False)
)
fig_marlboro_brands = px.bar(
    co_brands.head(15),
    x='co_occurrences',
    y='brandName',
    orientation='h',
    title='Brands Purchased with Marlboro',
    labels={'co_occurrences': 'Number of Shared Transactions'}
)
fig_marlboro_brands.show()

co_categories = (
    marlboro_baskets.groupby('category')
                    .agg(co_occurrences=('InteractionID', 'nunique'))
                    .reset_index()
                    .sort_values('co_occurrences', ascending=False)
)
fig_marlboro_categories = px.bar(
    co_categories,
    x='category',
    y='co_occurrences',
    title='Categories Purchased with Marlboro'
)
fig_marlboro_categories.update_layout(xaxis_tickangle=45)
fig_marlboro_categories.show()



## Laundry Analysis


In [96]:
laundry_keywords = ['laundry', 'detergent', 'surf', 'tide', 'breeze', 'downy', 'perla']

def is_laundry(row):
    text = ' '.join([
        str(row.get('category', '')),
        str(row.get('brandName', '')),
        str(row.get('productName', ''))
    ]).lower()
    return any(keyword in text for keyword in laundry_keywords)

laundry_df = items_df[items_df.apply(is_laundry, axis=1)].copy()

if not laundry_df.empty:
    laundry_df['hour'] = laundry_df['TransactionDate'].dt.hour
    laundry_df['weekday'] = laundry_df['TransactionDate'].dt.day_name()

    laundry_time = (
        laundry_df.groupby('hour')
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_laundry_time = px.line(
        laundry_time,
        x='hour',
        y='total_qty',
        markers=True,
        title='Laundry Purchases by Hour'
    )
    fig_laundry_time.show()

    laundry_day_avg = (
        laundry_df.groupby('weekday')
                  .agg(avg_qty=('quantity', 'mean'))
                  .reset_index()
    )
    fig_laundry_day = px.bar(
        laundry_day_avg,
        x='weekday',
        y='avg_qty',
        title='Laundry Avg Quantity by Weekday'
    )
    fig_laundry_day.show()

    laundry_brand = (
        laundry_df.groupby('brandName')
                  .agg(transactions=('InteractionID', 'nunique'),
                       avg_qty=('quantity', 'mean'))
                  .reset_index()
                  .sort_values('transactions', ascending=False)
    )
    fig_laundry_brand = px.scatter(
        laundry_brand,
        x='transactions',
        y='avg_qty',
        size='transactions',
        color='brandName',
        title='Laundry Brands — Volume vs Avg Qty'
    )
    fig_laundry_brand.show()

    laundry_gender = (
        laundry_df.groupby('gender_clean')
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_laundry_gender = px.bar(
        laundry_gender,
        x='gender_clean',
        y='total_qty',
        title='Laundry Qty by Gender'
    )
    fig_laundry_gender.show()

    laundry_age = (
        laundry_df.dropna(subset=['age_bucket'])
                  .groupby('age_bucket')
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_laundry_age = px.bar(
        laundry_age,
        x='age_bucket',
        y='total_qty',
        title='Laundry Qty by Age Bucket'
    )
    fig_laundry_age.show()

    laundry_gender_brand = (
        laundry_df.groupby(['gender_clean', 'brandName'])
                  .agg(total_qty=('quantity', 'sum'))
                  .reset_index()
    )
    fig_laundry_gender_brand = px.bar(
        laundry_gender_brand,
        x='brandName',
        y='total_qty',
        color='gender_clean',
        barmode='group',
        title='Laundry Brands by Gender'
    )
    fig_laundry_gender_brand.update_layout(xaxis_tickangle=45)
    fig_laundry_gender_brand.show()

    # Surf-focused baskets
    surf_txn_ids = laundry_df[laundry_df['brandName'].str.contains('surf', case=False, na=False)]['InteractionID'].unique()
    surf_baskets = laundry_df[laundry_df['InteractionID'].isin(surf_txn_ids)]
    surf_pairs = (
        surf_baskets[~surf_baskets['brandName'].str.contains('surf', case=False, na=False)]
                    .groupby('brandName')
                    .agg(co_occurrences=('InteractionID', 'nunique'))
                    .reset_index()
                    .sort_values('co_occurrences', ascending=False)
    )
    fig_surf = px.bar(
        surf_pairs.head(15),
        x='co_occurrences',
        y='brandName',
        orientation='h',
        title='Items Purchased with Surf'
    )
    fig_surf.show()
else:
    print('No laundry records detected — adjust keyword list if needed.')



/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/2972674378.py:76: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Notes for Metabase Implementation

- Each Plotly figure corresponds to a potential Metabase card; replicate the grouping logic (gender, daypart, basket bands, etc.) with the same aggregations.
- The helper columns (`gender_clean`, `age_bucket`, `basket_band`, `weekday_type`, etc.) can be materialized in SQL or a view to keep Metabase queries lightweight.
- Nested JSON fields (`totals_voice`, `transactionContext_voice`, `items_voice`) are flattened into `totals_*`, `context_*`, and `items_df`; mirror this flattening when building warehouse tables so filters remain fast.
- Daypart, payday windows, and round-price checks rely on simple CASE expressions, making them easy to port into SQL views feeding Metabase.


In [97]:
# --- Export `final_df` and `items_df` to Supabase via project URL/key ---

import os
from typing import List, Dict
from dotenv import load_dotenv
from pandas.api.types import is_datetime64_any_dtype
from datetime import date, datetime

load_dotenv()

try:
    from supabase import create_client, Client
except ImportError as e:
    raise ImportError(
        "supabase-py is not installed. Run `pip install supabase` in your environment."
    ) from e

# Recommended: set these as env vars instead of hardcoding
SUPABASE_URL: str = os.getenv("SUPABASE_URL")
SUPABASE_KEY: str = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

TRANSACTIONS_TABLE = "twba_transactions"
ITEMS_TABLE = "twba_items"

BATCH_SIZE = 5000


def upload_dataframe(df, table_name: str, pk: str | None = None):
    """Upload a DataFrame to Supabase, converting datetimes to ISO strings.

    Supabase (via the JS/HTTP API) expects JSON-serializable types, so we
    convert any datetime-like columns to string before sending.

    If a primary key `pk` is provided, rows are de-duplicated on that
    column before upload to avoid "ON CONFLICT ... cannot affect row a
    second time" errors when multiple rows share the same PK in a batch.
    """
    df_to_send = df.copy()

    # If we have a PK, drop duplicate keys in the dataframe itself
    if pk and pk in df_to_send.columns:
        df_to_send = df_to_send.drop_duplicates(subset=[pk])

    # Convert any datetime-like columns (including timezone-aware) to ISO8601 strings
    for col in df_to_send.columns:
        if is_datetime64_any_dtype(df_to_send[col]):
            df_to_send[col] = df_to_send[col].astype(str)

    # Also convert any Python date/datetime objects in non-datetime64 columns
    df_to_send = df_to_send.applymap(
        lambda v: v.isoformat() if isinstance(v, (date, datetime)) else v
    )

    records: List[Dict] = df_to_send.to_dict(orient="records")
    total = len(records)
    for start in range(0, total, BATCH_SIZE):
        batch = records[start : start + BATCH_SIZE]
        q = supabase.table(table_name)
        # If you defined a primary key (e.g. InteractionID), upsert avoids duplicates
        if pk:
            q = q.upsert(batch, on_conflict=pk)
        else:
            q = q.insert(batch)
        q.execute()

    print(f"Uploaded {total} rows into {table_name}.")


# Ensure integer-like columns are clean integers to match Postgres schema
if "Age" in final_df.columns:
    final_df["Age"] = (
        pd.to_numeric(final_df["Age"], errors="coerce")
        .round()
        .astype("Int64")
    )

if "Age" in items_df.columns:
    items_df["Age"] = (
        pd.to_numeric(items_df["Age"], errors="coerce")
        .round()
        .astype("Int64")
    )

if "quantity" in items_df.columns:
    items_df["quantity"] = (
        pd.to_numeric(items_df["quantity"], errors="coerce")
        .round()
        .astype("Int64")
    )

# Delete existing rows from tables to prevent duplicates
print(f"Deleting existing rows from {TRANSACTIONS_TABLE}...")
supabase.table(TRANSACTIONS_TABLE).delete().neq("InteractionID", "").execute()
print(f"Deleted existing rows from {TRANSACTIONS_TABLE}.")

print(f"Deleting existing rows from {ITEMS_TABLE}...")
supabase.table(ITEMS_TABLE).delete().neq("InteractionID", "").execute()
print(f"Deleted existing rows from {ITEMS_TABLE}.")

# Upload transaction-level table
upload_dataframe(final_df, TRANSACTIONS_TABLE, pk="InteractionID")

# For items, restrict to the columns defined in the twba_items DDL
items_df_for_db = items_df[[
    "InteractionID",
    "TransactionDate",
    "gender_clean",
    "age_bucket",
    "Age",
    "transactionContext_paymentMethod_voice",
    "totals_totalAmount_voice",
    "totalPrice",
    "unitPrice",
    "quantity",
    "category",
    "brandName",
    "productName",
    "sku",
    "timeofday_segment",
    "txn_weekday",
    "round_price_flag",
]].copy()

# Upload item-level table
upload_dataframe(items_df_for_db, ITEMS_TABLE)


Deleting existing rows from twba_transactions...
Deleted existing rows from twba_transactions.
Deleting existing rows from twba_items...
Deleted existing rows from twba_items.


/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/2385915186.py:52: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



Uploaded 10362 rows into twba_transactions.


/var/folders/1j/kqqtkbh15_39_wn6j6mnhcz80000gn/T/ipykernel_55980/2385915186.py:52: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



Uploaded 22613 rows into twba_items.


In [98]:
items_df.head()

,InteractionID,TransactionDate,gender_clean,age_bucket,Age,transactionContext_paymentMethod_voice,totals_totalAmount_voice,brandName,productName,genericName,...,brandConfidence,suggestedBrands,notes,customerRequest.requestType,customerRequest.specificBrand,customerRequest.pointedToProduct,customerRequest.acceptedSuggestion,timeofday_segment,txn_weekday,round_price_flag
0,000cef26-91c2-494e-8d81-fb602dc28906,2025-05-29 04:38:52+00:00,Male,35-44,36,cash,336.00,Pepsi,Pepsi Regular 1.5L,Product,...,0.82,None,STT: 'pepsi isa café puro',branded,True,False,False,Late Night (10p-5a),Thursday,None
1,000cef26-91c2-494e-8d81-fb602dc28906,2025-05-29 04:38:52+00:00,Male,35-44,36,cash,336.00,Café Puro,Café Puro Barako,Product,...,0.88,None,"STT: 'pepsi isa café puro dalawa selecta, pabi...",branded,True,False,False,Late Night (10p-5a),Thursday,None
2,000cef26-91c2-494e-8d81-fb602dc28906,2025-05-29 04:38:52+00:00,Male,35-44,36,cash,336.00,Selecta,Selecta Fortified Milk Powder,Product,...,0.82,None,"STT: 'café puro dalawa selecta, pabili po'",branded,True,False,False,Late Night (10p-5a),Thursday,None
3,000f9d0e-5c9b-4f5d-92e9-3bd60de578ce,2025-10-04 13:39:31+00:00,Female,35-44,39,cash,42.75,Nescafé,Nescafé 3in1 Creamy Latte,Product,...,0.82,None,STT: 'Nescafé malamig Kopiko isang',branded,True,False,False,Afternoon (12p-6p),Saturday,None
4,000f9d0e-5c9b-4f5d-92e9-3bd60de578ce,2025-10-04 13:39:31+00:00,Female,35-44,39,cash,42.75,Kopiko,Kopiko 78°C Coffee Latte,Product,...,0.82,None,STT: 'Nescafé malamig Kopiko isang Nido cold',branded,True,False,False,Afternoon (12p-6p),Saturday,None
